In [1]:
import torch
import numpy as np
import torch.nn as nn
import os
import time
from torch.utils.data import Dataset, DataLoader
from human_body_prior.body_model.body_model import BodyModel
from human_body_prior.tools.omni_tools import copy2cpu as c2c
from human_body_prior.tools.rotation_tools import aa2matrot,matrot2aa,local2global_pose
import random
from utils import utils_transform
from models.network import AvatarPoser as net
from scipy import signal

import glob
from IPython import embed
import time
import copy
import pickle
from human_body_prior.body_model.body_model import BodyModel

c:\Users\rnwns\anaconda3\envs\pytorch_geometric\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Network

In [2]:
class AvatarPoser(nn.Module):
    def __init__(self, input_dim, output_dim, num_layer, embed_dim, nhead, body_model, device):
        super(AvatarPoser, self).__init__()

        self.linear_embedding = nn.Linear(input_dim,embed_dim)

        encoder_layer = nn.TransformerEncoderLayer(embed_dim, nhead=nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layer)        

        self.stabilizer = nn.Sequential(
                            nn.Linear(embed_dim, 256),
                            nn.ReLU(),
                            nn.Linear(256, 6)
            )
        self.joint_rotation_decoder = nn.Sequential(
                            nn.Linear(embed_dim, 256),
                            nn.ReLU(),
                            nn.Linear(256, 126)
            )

        self.body_model = body_model

    @staticmethod
    def fk_module(global_orientation, joint_rotation, body_model):

        global_orientation = utils_transform.sixd2aa(global_orientation.reshape(-1,6)).reshape(global_orientation.shape[0],-1).float()
        joint_rotation = utils_transform.sixd2aa(joint_rotation.reshape(-1,6)).reshape(joint_rotation.shape[0],-1).float()
        body_pose = body_model(**{'pose_body':joint_rotation, 'root_orient':global_orientation})
        joint_position = body_pose.Jtr

        return joint_position


    @staticmethod
    def ik_module(smpl, smpl_jids, target_pose_ids, target_3ds,
                         body_pose = None, global_orient = None, transl = None, learning_rate=1e-1, n_iter=5):
        target_3ds = target_3ds.view(1, -1, 3)
        body_pose_sub = torch.tensor(body_pose[:, target_pose_ids],requires_grad = True)
        opti_param = [body_pose_sub]
        optimiser = torch.optim.Adam(opti_param, lr = learning_rate)

        for i in range(n_iter):
            body_pose[:, target_pose_ids] = body_pose_sub
            out = smpl(**{'pose_body':body_pose, 'root_orient':global_orient, 'trans': transl})
            j_3ds = out.Jtr.view(1, -1, 3)
            loss = torch.mean(torch.sqrt(torch.sum(torch.square(j_3ds[:, smpl_jids].squeeze()-target_3ds)[:,[20,21],:],axis=-1)))

            optimiser.zero_grad()
            loss.backward(retain_graph=True)
            optimiser.step()
            body_pose = body_pose.detach()
        return body_pose


    def forward(self, input_tensor, do_fk = True):

#        embed()
        x = self.linear_embedding(input_tensor)
        x = x.permute(1,0,2)
        x = self.transformer_encoder(x)
        x = x.permute(1,0,2)[:, -1]

        global_orientation = self.stabilizer(x)
        joint_rotation = self.joint_rotation_decoder(x)
        if do_fk:
            joint_position = self.fk_module(global_orientation, joint_rotation, self.body_model)
            return global_orientation, joint_rotation, joint_position
        else:
            return global_orientation, joint_rotation


In [5]:
path_dir = './data_fps60/CMU/test/'
file_list = os.listdir(path_dir)
file_dir = os.path.join(path_dir,file_list[0])

data = np.load(file_dir, allow_pickle=True)

f = data['hmd_position_global_full_gt_list']
print(data.keys())

input_hmd  = f.reshape(f.shape[0], -1)[1:]
print(f.shape)
print(input_hmd.shape)

dict_keys(['rotation_local_full_gt_list', 'hmd_position_global_full_gt_list', 'body_parms_list', 'head_global_trans_list', 'framerate', 'gender', 'filepath', 'position_global_full_gt_world'])
torch.Size([2246, 54])
torch.Size([2245, 54])


In [7]:
# input_dim, output_dim, num_layer, embed_dim, nhead, body_model, device

'''    , "netG": {
      "net_type": "AvatarPoser" 
      , "num_layer": 3
      , "input_dim": 54        // input channel number # 36
      , "output_dim": 132       // ouput channel number   #132
      , "embed_dim": 256          // channel number of linear embedding for transformer
      , "nhead": 8          // 
      , "init_type": "kaiming_normal"         // "orthogonal" | "normal" | "uniform" | "xavier_normal" | "xavier_uniform" | "kaiming_normal" | "kaiming_uniform"
      , "init_bn_type": "uniform"         // "uniform" | "constant"
      , "init_gain": 0.2
    }
'''

pth_path = './model_zoo/avatarposer.pth'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
support_dir = "./support_data/"
subject_gender = "male"
bm_fname = os.path.join(support_dir, 'body_models/smplh/{}/model.npz'.format(subject_gender))
dmpl_fname = os.path.join(support_dir, 'body_models/dmpls/{}/model.npz'.format(subject_gender))
num_betas = 16 # number of body parameters
num_dmpls = 8 # number of DMPL parameters
body_model = BodyModel(bm_fname=bm_fname, num_betas=num_betas, num_dmpls=num_dmpls, dmpl_fname=dmpl_fname).to(device)

path_dir = './data_fps60/MPI_HDM05/test/'
file_list = os.listdir(path_dir)

file_dir = os.path.join(path_dir,file_list[0])
print(file_dir)

start = time.time()

for data in file_list:
    file_dir = os.path.join(path_dir, data)
    data = np.load(file_dir, allow_pickle=True)

    test_loader = DataLoader(data['hmd_position_global_full_gt_list'], batch_size=1,shuffle=False, num_workers=1, drop_last=False, pin_memory=True)
    model = AvatarPoser(input_dim=54, output_dim=132, num_layer=3, embed_dim=256, nhead=8, body_model=body_model,device=device)
    model.load_state_dict(torch.load(pth_path))
    model.to(device)
    global_orientation = []
    joint_rotation = []
    joint_position = []

    for index, test_data in enumerate(test_loader) :
        test_data = test_data.to('cuda').float()
        test_data = test_data.view(1,1,54)

        x = model(test_data)

        global_orientation.append(x[0].cpu().detach().numpy())
        joint_rotation.append(x[1].cpu().detach().numpy())
        joint_position.append(x[2].cpu().detach().numpy())
        
        print("idx : {}".format(index))

end = time.time()

runtime = end - start
print("Runtime is : {:.3f}".format(runtime))

./data_fps60/MPI_HDM05/test/1.pkl


FileNotFoundError: [Errno 2] No such file or directory: './data_fps60/MPI_HDM05/test/1'

In [ ]:
global_orientation = np.array(global_orientation).squeeze()
joint_rotation = np.array(joint_rotation).squeeze()
joint_position = np.array(joint_position).squeeze()

In [ ]:
npz_path = './test_data.npz'
np.savez(npz_path, global_orientation = global_orientation, joint_rotation = joint_rotation, joint_position = joint_position)

In [ ]:
what = np.load(npz_path)

for keys in what:
    print(keys)
    print(what[keys].shape)

print(x[0].shape, x[1].shape, x[2].shape,)